# भाग 9 - इंट्रो टू एन्क्रिप्टेड प्रोग्राम

यावर विश्वास ठेवा किंवा नाही, एनक्रिप्टेड डेटासह गणना करणे शक्य आहे. दुऱ्या शब्दांत, प्रोग्राम चालवणे शक्य आहे जेथे **प्रोग्राममधील सर्व चल एन्क्रिप्ट केलेले आहेत!**

या ट्यूटोरियल मध्ये आपण एनक्रिप्टेड संगणनाची मूलभूत साधने पार पाडणार आहोत. विशेषतः आपण सिक्युर मल्टी-पार्टी कंप्यूटेशन नावाच्या लोकप्रिय दृष्टिकोनावर लक्ष केंद्रित करणार आहोत. या पाठात आपण एन्क्रिप्टेड कॅल्क्युलेटर कसे तयार करावे जे एनक्रिप्टेड नंबरवर गणना करू शकतात.

लेखक:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - Github: [@LaRiffle](https://github.com/Laiffiff)

संदर्भ:
- Morten Dahl - [ब्लॉग](https://mortendahl.github.io) - Twitter: [@mortendahlcs](https://twitter.com/mortendahlcs)


अनुवादक/संपादक:
- Krunal Kshirsagar - Twitter: [@krunal_wrote](https://twitter.com/krunal_wrote) - Github: [@Noob-can-Compile](https://github.com/Noob-can-Compile)

# चरण 1: सिक्युर मल्टी-पार्टी कंप्यूटेशन वापरुन एनक्रिप्शन करणे

SMPC पहिल्या दृष्टीक्षेपात "encryption" चा एक विलक्षण प्रकार आहे. व्हेरिएबलला एनक्रिप्ट करण्यासाठी सार्वजनिक / खाजगी की वापरण्याऐवजी, प्रत्येक मूल्य एकाधिक `shares` मध्ये विभागले जाते, त्यातील प्रत्येक खाजगी की प्रमाणे कार्य करते. सामान्यत: हे `shares` भाग 2 किंवा अधिक _owners_मध्ये वितरीत केले जातील. अशाप्रकारे, व्हेरिएबल डिक्रिप्ट करण्यासाठी, सर्व मालकांनी डिक्रिप्शनला परवानगी देण्याचे मान्य केले पाहिजे. थोडक्यात, प्रत्येकाकडे खासगी की असते.

### Encrypt()

तर, चला असे म्हणा की आपल्याला व्हेरिएबल x "एनक्रिप्ट" करायचा आहे, असे आपण पुढील मार्गाने करू शकतो.

 > एंक्रिप्शन फ्लोट्स किंवा वास्तविक संख्या वापरत नाही, परंतु गणिताच्या जागेवर [integer quotient ring](http://mathworld.wolfram.com/QuotientRing.html) म्हणतात, जो मुळात `0` आणि `Q-1` दरम्यान पूर्णांक असतो, जेथे `Q` प्राइम आणि "पुरेशी" असते जेणेकरून आपल्या प्रयोगांमध्ये वापरलेल्या सर्व संख्या समाविष्ट होतील. प्रॅक्टिसमध्ये, 'x' पूर्णांकचे मूल्य देऊन, आपण रिंगमध्ये फिट होण्यासाठी `x % Q` देतो. (म्हणूनच आपण `x '> Q` हा अंक वापरणे टाळत आहोत).


In [1]:
Q = 1234567891011

In [2]:
x = 25

In [3]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [4]:
encrypt(x)

(890804432397, -2305631655, 346069090294)

आपण येथे पाहू शकता की आम्ही आपल्या व्हेरिएबल `x` ला 3 वेगवेगळ्या शेअर्समध्ये विभागले आहे, जे 3 वेगवेगळ्या मालकांना पाठवले जाऊ शकतात.

### Decrypt()

जर आपल्याला हे 3 समभाग डीक्रिप्ट करायचे असतील तर आपल्याला फक्त त्यांना एकत्र करुन निकालाचे मॉड्यूलस (mod Q) घेऊ शकू.

In [5]:
def decrypt(*shares):
    return sum(shares) % Q

In [6]:
a,b,c = encrypt(25)

In [7]:
decrypt(a, b, c)

25

महत्त्वाचे म्हणजे लक्षात घ्या की जर आपण केवळ दोन शेअर्ससह (shares) डिक्रिप्ट करण्याचा प्रयत्न केला तर डिक्रिप्शन कार्य करत नाही!

In [8]:
decrypt(a, b)

778460474681

अशा प्रकारे, मूल्य डीक्रिप्ट करण्यासाठी आपल्याला सर्व मालकांच्या  सहभागाची आवश्यकता आहे. हे अशा प्रकारे आहे की `shares` खाजगी की प्रमाणे कार्य करतात, त्यातील सर्व मूल्य डीक्रिप्ट करण्यासाठी उपस्थित असणे आवश्यक आहे.

# चरण 2: SMPC वापरुन मूलभूत अंकगणित

तथापि, **व्हेरिएबल्स अजूनही एनक्रिप्ट केले जात असताना** सुरक्षित बहु-पक्षीय संगणनांची खरोखर अपवादात्मक मालमत्ता गणना करण्याची क्षमता आहे. खाली साधे भर दाखवू.

In [9]:
x = encrypt(25)
y = encrypt(5)

In [10]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [11]:
decrypt(*add(x,y))

30

### यश !!!

आणि तुम्हाला ते मिळालेच! जर प्रत्येक कामगार (स्वतंत्रपणे) त्यांचे शेअर्स एकत्र जोडत असेल तर परिणामी शेअर्स योग्य मूल्यावर (25 + 5 == 30) डिक्रिप्ट होतील.

जसे हे निष्पन्न होते, SMPC प्रोटोकॉल अस्तित्त्वात आहेत जे पुढील क्रियांसाठी या एनक्रिप्टेड संगणनास अनुमती देऊ शकतात:
- जोड (जे आपण आत्ताच पाहिले आहे)
- गुणाकार
- तुलना

आणि या मूलभूत आदिमांचा वापर करून आपण अनियंत्रित गणना करू शकतो !!!

पुढील विभागात, आपण ही ऑपरेशन्स करण्यासाठी PySyft लायब्ररी कशी वापरावी हे शिकणार आहोत!

# चरण 3: PySyft वापरुन SMPC

मागील भागात, आपण SMPC च्या आसपास काम करणारे काही मूलभूत अंतर्ज्ञान अधोरेखित केले. तथापि, प्रत्यक्षात आपले एनक्रिप्टेड प्रोग्राम्स लिहिताना आपण स्वतः सर्व आदिम ऑपरेशन्स स्वतःच लिहायच्या नसतात. तर, या विभागात आपण PySyft चा वापर करून एनक्रिप्टेड कंप्यूटेशन कसे करावे याबद्दल मूलभूत गोष्टी जाणून घेणार आहोत. विशेषत: आपण पूर्वी नमूद केलेले 3 आदिम कसे करावे यावर लक्ष केंद्रित करणार आहोत: जोड, गुणाकार आणि तुलना.

प्रथम, आपल्याला काही आभासी कामगार (Virtual Workers) तयार करण्याची आवश्यकता आहे (जे, आशा आहे की आपण आता आमच्या मागील शिकवण्यांसह परिचित आहात).

In [12]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

### मूलभूत एन्क्रिप्शन / डिक्रिप्शन

एन्क्रिप्शन कोणत्याही PySyft टेन्सर आणि .share() ला कॉल करून घेण्याइतकेच सोपे आहे. डिक्रिप्शन सामायिक व्हेरिएबलवर .get () वर कॉल करण्याइतके सोपे आहे. 

In [13]:
x = torch.tensor([25])

In [14]:
x

tensor([25])

In [15]:
encrypted_x = x.share(bob, alice, bill)

In [16]:
encrypted_x.get()

tensor([25])

### एन्क्रिप्ट आणकांचे आत्मपरीक्षण करणे

एन्क्रिप्ट आणकांचे आत्मपरीक्षण करणे
जर आपण Bob, Alice आणि Bill चे कामगारांकडे पाहिले तर आपण तयार झालेले शेअर्स पाहू शकतो.

In [17]:
bob._objects

{}

In [18]:
x = torch.tensor([25]).share(bob, alice, bill)

In [19]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([3212861001891376707])

In [20]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([61371170032936135])

In [21]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([1337453846503075087])

आणि जर आपल्याला पाहिजे असेल तर आपण पूर्वी बोललेल्या समान पध्दतीचा वापर करून ही मूल्ये डीक्रिप्ट करु शकता !!!

In [22]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

जसे आपण पाहू शकता, जेव्हा आपण `.share()` कॉल केला तेव्हा त्यास फक्त तीन शेअर्समध्ये मूल्य विभागले आणि प्रत्येक पक्षाला एक वाटा पाठविला!

# एन्क्रिप्टेड अंकगणित

आणि आता तुम्ही पाहता की आपण मूळ मूल्यांवर अंकगणित करू शकतो! एपीआय तयार केले आहे जेणेकरून आपण PyTorch टेन्सर सामान्य करण्यासारखे गणित करू शकतो.

In [23]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [24]:
z = x + y
z.get()

tensor([30])

In [25]:
z = x - y
z.get()

tensor([20])

# एन्क्रिप्ट गुणाकार

गुणाकार करण्यासाठी आपल्याला एक अतिरिक्त पक्षाची आवश्यकता आहे जी सलग यादृच्छिक संख्या निर्माण करण्यासाठी जबाबदार असेल (आणि इतर कोणत्याही पक्षांसह एकत्र येत नाही). आपण या व्यक्तीस "क्रिप्टो प्रदाता"("crypto provider") म्हणतो. सर्व सखोल हेतूंसाठी, क्रिप्टो प्रदाता फक्त एक अतिरिक्त व्हर्च्युअल वर्कर (VirtualWorker) आहे, परंतु हे कबूल करणे महत्वाचे आहे की क्रिप्टो प्रदाता "मालक" नाही ज्यात तो / तिचा शेअर्सचा मालक नाही परंतु कोणावर विश्वास ठेवला जाऊ शकतो विद्यमान कोणत्याही भागधारकांशी करार करीत नाही.

In [26]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [27]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [28]:
# multiplication

z = x * y
z.get()

tensor([125])

आपण मॅट्रिक्स गुणाकार देखील करू शकता

In [29]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [30]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

# एन्क्रिप्टेड तुुलना

खाजगी मूल्यांमध्ये खासगी तुलना करणे देखील शक्य आहे. आम्ही येथे SecureNN प्रोटोकॉलवर अवलंबून आहोत, ज्याचा तपशील [येथे](https://eprint.iacr.org/2018/442.pdf) मिळू शकतो. तुलनाचा परिणाम देखील एक खाजगी सामायिक टेन्सर आहे.

In [31]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [32]:
z = x > y
z.get()

tensor([1])

In [33]:
z = x <= y
z.get()

tensor([0])

In [34]:
z = x == y
z.get()

tensor([0])

In [35]:
z = x == y + 20
z.get()

tensor([1])

आपण max ऑपरेशन्स देखील करू शकता

In [36]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [37]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])

# अभिनंदन !!! - समुदायात सामील होण्याची वेळ आली!


हे नोटबुक ट्यूटोरियल पूर्ण केल्याबद्दल अभिनंदन! आपण याचा आनंद घेत असल्यास आणि एआय(AI) आणि एआय सप्लाय चेन  (डेटा) च्या विकेंद्रित(Decentralized) मालकीच्या गोपनीयतेच्या संरक्षणाच्या दिशेने चळवळीत सामील होऊ इच्छित असाल तर आपण हे खालील प्रकारे करू शकता!

### Pysyft ला Github वर Star करा!

आमच्या समुदायाला मदत करण्याचा सर्वात सोपा मार्ग म्हणजे फक्त गिटहब(GitHub) रेपो(Repo) तारांकित(Star) करणे! हे आम्ही तयार करीत असलेल्या छान साधनांविषयी जागरूकता वाढविण्यास मदत करते.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### आमच्या Slack मध्ये सामील व्हा!


नवीनतम प्रगतीवर अद्ययावत राहण्याचा उत्तम मार्ग म्हणजे आमच्या समुदायामध्ये सामील होणे! आपण [http://slack.openmined.org](http://slack.openmined.org) येथे फॉर्म भरुन तसे करू शकता.

### एका कोड प्रोजेक्टमध्ये सामील व्हा!

आमच्या समुदायामध्ये योगदानाचा उत्तम मार्ग म्हणजे कोड योगदानकर्ता बनणे! कोणत्याही वेळी आपण (PySyft GitHub Issues Page) वर  जाऊ शकता आणि "Project" साठी फिल्टर करू शकता. हे आपण कोणत्या प्रकल्पांमध्ये सामील होऊ शकता याबद्दल विहंगावलोकन देणारी सर्व उच्च स्तरीय तिकिटे दर्शवेल! आपण एखाद्या प्रकल्पात सामील होऊ इच्छित नसल्यास, परंतु आपण थोडं कोडिंग करू इच्छित असाल तर आपण "good first issues" म्हणून चिन्हांकित गिटहब(GitHub) अंक शोधून आणखी "one off" मिनी-प्रकल्प(mini project) शोधू शकता.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करा

आपल्याकडे आमच्या कोडेबेसमध्ये योगदान देण्यास वेळ नसल्यास, परंतु तरीही आपल्याला समर्थन द्यावयाचे असल्यास आपण आमच्या मुक्त संग्रहात बॅकर देखील होऊ शकता. सर्व देणगी आमच्या वेब होस्टिंग आणि हॅकॅथॉन आणि मेटअप्स सारख्या इतर सामुदायिक खर्चाकडे जातात!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)